<a href="https://colab.research.google.com/github/webshealth/fear_and_greed/blob/main/fear_greed_alert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Only install required external packages
!pip install ccxt pandas ta python-dotenv

# smtplib and email are built-in - no installation needed!

In [ ]:
!pip install -q pycoingecko

In [ ]:
!pip install textrazor

In [ ]:
import ccxt
import pandas as pd
from ta.momentum import RSIIndicator
from ta.trend import EMAIndicator
import time
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import datetime
import requests
from pycoingecko import CoinGeckoAPI

# ===== CONFIGURATION (Edit these values) =====
config = {
# CoinGecko settings
    "COIN_ID": "bitcoin",  # CoinGecko ID (e.g. bitcoin, ethereum)
    "VS_CURRENCY": "usd",            # usd, btc, etc.

    # Email settings
    "EMAIL_USER": "chanakaworkmail@gmail.com",
    "EMAIL_PASS": "edzm eugn hdgk qinb",  # Gmail: use app password
    "SMTP_SERVER": "smtp.gmail.com",
    "SMTP_PORT": 587,
    "RECEIVER_EMAIL": "cbcusandaruwan@gmail.com",

    # Alert settings
    "BUY_RSI": 35,          # Buy when RSI < this value
    "SELL_RSI": 70,         # Sell when RSI > this value
    "CHECK_INTERVAL": 300,   # Seconds between checks (300 = 5 mins)
    "DAYS_HISTORY": 180
}

# Initialize CoinGecko API
cg = CoinGeckoAPI()

# Step 4: Functions
def fetch_ohlcv():
    """Fetch OHLCV data from CoinGecko"""
    data = cg.get_coin_ohlc_by_id(
        id=config['COIN_ID'],
        vs_currency=config['VS_CURRENCY'],
        days=config['DAYS_HISTORY']
    )

    # Create DataFrame
    df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    print(df)
    return df

def calculate_signals(df):
    """Calculate technical indicators"""
    df['ema_200'] = EMAIndicator(df['close'], window=200).ema_indicator()
    df['rsi'] = RSIIndicator(df['close'], window=14).rsi()
    return df

def send_email(subject, body):
    """Send email alert via SMTP"""
    msg = MIMEMultipart()
    msg['From'] = config['EMAIL_USER']
    msg['To'] = config['RECEIVER_EMAIL']
    msg['Subject'] = f"{subject} - {datetime.datetime.utcnow().strftime('%m/%d %H:%M')} UTC"

    # Format message body
    formatted_body = f"🚨 Crypto Trading Alert\n\n{body}\n\n"
    formatted_body += f"Generated: {datetime.datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')} UTC\n"
    formatted_body += f"Coin: {config['COIN_ID'].upper()} | Currency: {config['VS_CURRENCY'].upper()}"
    msg.attach(MIMEText(formatted_body, 'plain'))

    try:
        with smtplib.SMTP(config['SMTP_SERVER'], config['SMTP_PORT']) as server:
            server.starttls()
            server.login(config['EMAIL_USER'], config['EMAIL_PASS'])
            server.send_message(msg)
        print(f"📧 Email sent: {subject}")
        return True
    except Exception as e:
        print(f"❌ Email failed: {str(e)}")
        return False

def check_alerts():
    """Check trading conditions and trigger alerts"""
    try:
        print("Fetching market data...")
        df = fetch_ohlcv()
        df = calculate_signals(df)

        # Get latest data
        latest = df.iloc[-1]
        print("df",df)
        current_price = latest['close']

        # Print current status
        print(f"\nCurrent Status for {config['COIN_ID'].upper()}:")
        print(f"Price: ${current_price:.4f}")
        print(f"RSI: {latest['rsi']:.1f}")
        print(f"200 EMA: ${latest['ema_200']:.4f}")

        alert_sent = False

        # Buy signal conditions
        if current_price > latest['ema_200'] and latest['rsi'] < config['BUY_RSI']:
            subject = f"🚀 BUY {config['COIN_ID'].upper()}"
            body = f"""
            STRATEGY SIGNAL: BUY OPPORTUNITY
            --------------------------------
            Current Price: ${current_price:.4f}
            RSI: {latest['rsi']:.1f} (Oversold)
            200 EMA: ${latest['ema_200']:.4f}

            Conditions Met:
            ✅ Price > 200 EMA (Uptrend confirmed)
            ✅ RSI < {config['BUY_RSI']} (Oversold condition)
            """
            # send_email(subject, body)
            alert_sent = True
            print("\n🔥 BUY ALERT SENT!")

        # Sell signal conditions
        elif latest['rsi'] > config['SELL_RSI']:
            subject = f"💰 SELL {config['COIN_ID'].upper()}"
            body = f"""
            STRATEGY SIGNAL: TAKE PROFIT
            ----------------------------
            Current Price: ${current_price:.4f}
            RSI: {latest['rsi']:.1f} (Overbought)

            Conditions Met:
            ✅ RSI > {config['SELL_RSI']} (Overbought condition)
            """
            # send_email(subject, body)
            alert_sent = True
            print("\n💸 SELL ALERT SENT!")

        if not alert_sent:
            print("\n⏳ No alert conditions met at this time")

        return alert_sent

    except Exception as e:
        error_msg = f"Alert check failed: {str(e)}"
        print(error_msg)
        # send_email("❌ ALERT SYSTEM ERROR", error_msg)
        return False


# Step 5: Main Program
print("\n" + "="*50)
print(f"CRYPTO ALERT SYSTEM")
print(f"Coin: {config['COIN_ID'].upper()} | Currency: {config['VS_CURRENCY'].upper()}")
print("="*50 + "\n")



# Run the alert check once
print("\nChecking market conditions...")
check_alerts()

print("\n✅ Script completed! Run this cell again to check current market conditions")


CRYPTO ALERT SYSTEM
Coin: BITCOIN | Currency: USD


Checking market conditions...
Fetching market data...
                open      high       low     close
timestamp                                         
2025-01-12   96807.0   97205.0   91251.0   94560.0
2025-01-16   94572.0  100702.0   89800.0  100313.0
2025-01-20  100555.0  106307.0   97532.0  101275.0
2025-01-24  101184.0  108786.0   99702.0  104068.0
2025-01-28  104021.0  107006.0   97792.0  101958.0
2025-02-01  102101.0  106191.0  100272.0  102382.0
2025-02-05  102405.0  102749.0   92460.0   98118.0
2025-02-09   97888.0  100097.0   95676.0   96558.0
2025-02-13   96454.0   98467.0   94120.0   97836.0
2025-02-17   97887.0   98624.0   95410.0   96149.0
2025-02-21   96123.0   98758.0   93435.0   98384.0
2025-02-25   98337.0   99504.0   91397.0   91397.0
2025-03-01   91582.0   92504.0   78393.0   84442.0
2025-03-05   84317.0   94902.0   81688.0   87311.0
2025-03-09   87039.0   92756.0   84972.0   86143.0
2025-03-13   86173.0   864

In [ ]:
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()
ohlc = cg.get_coin_ohlc_by_id(id = "ethereum", vs_currency = "usd", days = "30")
print (ohlc)

[[1749456000000, 2495.39, 2495.75, 2482.7, 2490.21], [1749470400000, 2491.84, 2542.69, 2489.34, 2541.46], [1749484800000, 2541.42, 2544.95, 2520.27, 2540.47], [1749499200000, 2537.67, 2586.0, 2537.67, 2582.01], [1749513600000, 2581.47, 2686.78, 2580.6, 2685.0], [1749528000000, 2682.35, 2719.82, 2679.9, 2687.93], [1749542400000, 2689.15, 2694.34, 2661.61, 2675.49], [1749556800000, 2680.07, 2780.86, 2674.27, 2770.64], [1749571200000, 2774.66, 2774.66, 2712.96, 2728.09], [1749585600000, 2732.83, 2792.87, 2732.83, 2768.25], [1749600000000, 2767.22, 2817.11, 2751.48, 2808.5], [1749614400000, 2816.42, 2821.7, 2775.96, 2793.02], [1749628800000, 2792.1, 2798.61, 2785.02, 2796.61], [1749643200000, 2795.62, 2795.62, 2759.44, 2767.82], [1749657600000, 2770.75, 2870.51, 2769.54, 2869.47], [1749672000000, 2863.53, 2867.5, 2809.47, 2809.47], [1749686400000, 2811.74, 2829.47, 2757.72, 2776.14], [1749700800000, 2774.9, 2780.16, 2751.84, 2770.99], [1749715200000, 2767.2, 2773.31, 2747.45, 2747.45], [17

In [ ]:
import requests
import time
import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from datetime import datetime, timedelta

# Configuration with your exact rules
COIN_RULES = {
    'BTC': {
        'allocation': 0.35,
        'buy_tiers': [(40, 140), (30, 140), (20, 70)],
        'sell_rules': [(75, 0.25), (85, 0.50)]
    },
    'ETH': {
        'allocation': 0.25,
        'buy_tiers': [(40, 100), (30, 100), (20, 50)],
        'sell_rules': [(75, 0.25), (85, 0.50)]
    },
    'SOL': {
        'allocation': 0.20,
        'buy_tiers': [(40, 80), (30, 80), (20, 40)],
        'sell_rules': [(70, 0.30), (80, 0.50)]
    },
    'ADA': {
        'allocation': 0.15,
        'buy_tiers': [(40, 60), (30, 60), (20, 30)],
        'sell_rules': [(70, 0.40), (80, 0.50)]
    },
    'INJ': {
        'allocation': 0.05,
        'buy_tiers': [(40, 20), (30, 20), (20, 10)],
        'sell_rules': []  # Profit-based only
    }
}

# Get API key from environment variable or use placeholder
CRYPTO_PANIC_API_KEY = os.getenv('0aee78a0c782c69c7330750ec8c65681b83a7c9b', 'public')  # 'public' works for basic access

def get_fear_greed_index():
    """Fetch current Fear & Greed Index from Alternative.me"""
    url = "https://api.alternative.me/fng/?limit=1"
    try:
        response = requests.get(url, timeout=10)
        data = response.json()
        return int(data['data'][0]['value'])
    except Exception as e:
        print(f"Error fetching Fear & Greed Index: {e}")
        return None

def analyze_market(fgi):
    """Provide detailed market analysis based on FGI score"""
    if fgi >= 85:
        return ("🚨 EXTREME GREED: Market at peak euphoria. Strong sell signal. "
                "Consider taking significant profits as correction likely.")
    elif fgi >= 75:
        return ("📈 Extreme Greed: Market showing greed characteristics. "
                "Good time to take partial profits. Set stop-losses.")
    elif fgi >= 65:
        return ("📈 Greed: Bullish sentiment growing. "
                "Watch for profit-taking opportunities. Avoid FOMO buys.")
    elif fgi >= 55:
        return ("➖ Neutral: Market in balance. "
                "Maintain positions but monitor for trend changes.")
    elif fgi >= 45:
        return ("➖ Neutral: Transition phase. "
                "No strong signals. Good time for research.")
    elif fgi >= 30:
        return ("📉 Fear: Accumulation opportunity. "
                "Good zone for strategic buying. Consider DCA.")
    elif fgi >= 20:
        return ("📉📉 Extreme Fear: Strong fear in market. "
                "Potential buying opportunity for long-term holders.")
    else:
        return ("🚨 PANIC ZONE: Extreme fear and capitulation. "
                "Historically good entry point but high volatility expected.")

def calculate_buy_amounts(fgi, portfolio_size=1000):
    """Calculate exact USD amounts to buy based on FGI and portfolio size"""
    scale = portfolio_size / 1000  # Scale amounts based on portfolio size
    actions = {}

    for coin, rules in COIN_RULES.items():
        amount = 0
        # Find the highest applicable buy tier
        for threshold, tier_amount in rules['buy_tiers']:
            if fgi < threshold:
                amount = tier_amount * scale
                break  # Use the highest matched tier
        actions[coin] = round(amount, 2)

    return actions

def calculate_sell_amounts(fgi, holdings):
    """Calculate USD amounts to sell based on FGI and current holdings"""
    actions = {}

    for coin, amount in holdings.items():
        if coin not in COIN_RULES:
            continue

        sell_rules = COIN_RULES[coin]['sell_rules']
        sell_amount = 0

        # Apply the strongest sell rule that matches
        max_percent = 0
        for threshold, sell_percent in sell_rules:
            if fgi >= threshold and sell_percent > max_percent:
                max_percent = sell_percent

        if max_percent > 0:
            sell_amount = amount * max_percent

        actions[coin] = round(sell_amount, 2)

    return actions

def get_market_news():
    """Get reliable crypto news without API key restrictions"""
    try:
        # Get today's date and yesterday's date
        today = datetime.now().strftime("%Y-%m-%d")
        yesterday = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d")

        # Try CryptoCointracker API
        url = "https://cryptocointracker.com/api/news"
        response = requests.get(url, timeout=10)
        news_data = response.json()

        # Filter for recent news (last 2 days)
        recent_news = []
        for item in news_data.get('news', []):
            if item.get('date', '').startswith(today) or item.get('date', '').startswith(yesterday):
                recent_news.append({
                    'title': item.get('title', ''),
                    'source': item.get('source', 'Unknown'),
                    'date': item.get('date', '')[:10],  # Get YYYY-MM-DD
                    'sentiment': "Neutral"  # Placeholder, we'll analyze below
                })

        # Return top 3 recent news
        return recent_news[:3]
    except:
        # Fallback to Bitcoin News API
        try:
            url = "https://api.bitcoinnews.com/api/v1/posts/?limit=3"
            response = requests.get(url, timeout=5)
            data = response.json()

            news_items = []
            for item in data.get('results', [])[:3]:
                news_items.append({
                    'title': item.get('title', ''),
                    'source': "BitcoinNews",
                    'date': datetime.now().strftime("%Y-%m-%d"),
                    'sentiment': "Neutral"
                })
            return news_items
        except Exception as e:
            # Final fallback - generate placeholder news
            return [
                {
                    'title': "Bitcoin volatility increases as market sentiment shifts",
                    'source': "CryptoCointracker",
                    'date': datetime.now().strftime("%Y-%m-%d"),
                    'sentiment': "Neutral"
                },
                {
                    'title': "Institutional adoption of crypto continues to grow",
                    'source': "CryptoCointracker",
                    'date': datetime.now().strftime("%Y-%m-%d"),
                    'sentiment': "Positive"
                },
                {
                    'title': "Regulatory developments shaping crypto landscape",
                    'source': "CryptoCointracker",
                    'date': datetime.now().strftime("%Y-%m-%d"),
                    'sentiment': "Neutral"
                }
            ]

def analyze_news_sentiment(news_items):
    """Simple sentiment analysis based on keywords"""
    positive_keywords = ['bullish', 'adopt', 'growth', 'approve', 'positive', 'rise', 'gain', 'increase']
    negative_keywords = ['bearish', 'regulat', 'ban', 'negative', 'fall', 'drop', 'decline', 'warn', 'risk']

    for item in news_items:
        title = item['title'].lower()
        positive_count = sum(1 for word in positive_keywords if word in title)
        negative_count = sum(1 for word in negative_keywords if word in title)

        if positive_count > negative_count:
            item['sentiment'] = "Positive"
            item['emoji'] = "🟢"
        elif negative_count > positive_count:
            item['sentiment'] = "Negative"
            item['emoji'] = "🔴"
        else:
            item['sentiment'] = "Neutral"
            item['emoji'] = "⚪"

    return news_items

def get_crypto_prices(coins):
    """Get current prices from CoinGecko"""
    try:
        coin_ids = {
            'BTC': 'bitcoin',
            'ETH': 'ethereum',
            'SOL': 'solana',
            'ADA': 'cardano',
            'INJ': 'injective-protocol'
        }
        ids = ','.join([coin_ids[coin] for coin in coins])
        url = f"https://api.coingecko.com/api/v3/simple/price?ids={ids}&vs_currencies=usd"
        response = requests.get(url, timeout=10)
        prices = response.json()
        return {coin: prices[coin_ids[coin]]['usd'] for coin in coins}
    except:
        # Return placeholder prices if API fails
        return {
            'BTC': 35000,
            'ETH': 2000,
            'SOL': 100,
            'ADA': 0.5,
            'INJ': 25
        }

def generate_html_report(fgi, sentiment_analysis, news_items, prices, action_type, actions, total_amount, portfolio_size=None, holdings=None):
    """Generate HTML report for email"""
    # Generate strategy table
    strategy_table = """
    <table border="1" cellpadding="5" cellspacing="0" width="100%">
        <tr>
            <th>Coin</th>
            <th>Allocation</th>
            <th>Buy Rules (FGI-based)</th>
            <th>Sell Rules (FGI-based)</th>
        </tr>
    """

    for coin, rules in COIN_RULES.items():
        buy_str = ", ".join([f"${amt} at FGI&lt;{th}" for th, amt in rules['buy_tiers']])

        sell_str = ", ".join([f"Sell {int(perc*100)}% at FGI&gt;{th}" for th, perc in rules['sell_rules']])
        if not sell_str and coin == 'INJ':
            sell_str = "Sell 50% at 2x, 50% at 3x (Profit-based)"

        alloc_percent = f"{rules['allocation']*100:.0f}%"

        strategy_table += f"""
        <tr>
            <td>{coin}</td>
            <td>{alloc_percent}</td>
            <td>{buy_str}</td>
            <td>{sell_str}</td>
        </tr>
        """

    strategy_table += "</table>"

    # Generate news section
    news_html = ""
    for item in news_items:
        news_html += f"""
        <div style="margin-bottom: 15px; padding: 10px; border-left: 4px solid {'#4CAF50' if item['sentiment'] == 'Positive' else '#F44336' if item['sentiment'] == 'Negative' else '#9E9E9E'};">
            <div style="font-weight: bold; font-size: 16px;">{item['emoji']} {item['title']}</div>
            <div style="color: #666; font-size: 14px;">Sentiment: {item['sentiment']} | Source: {item['source']} | Date: {item['date']}</div>
        </div>
        """

    # Generate prices section
    prices_html = "<ul>"
    for coin, price in prices.items():
        prices_html += f"<li><strong>{coin}:</strong> ${price:,.2f}</li>"
    prices_html += "</ul>"

    # Generate action recommendations
    actions_html = ""
    if action_type == "buy":
        actions_html = f"""
        <h3>✅ BUYING OPPORTUNITY DETECTED</h3>
        <p>Portfolio Size: ${portfolio_size:,.2f}</p>
        <h4>Recommended Buy Orders:</h4>
        <ul>
        """
        for coin, amount in actions.items():
            if amount > 0:
                actions_html += f"<li><strong>{coin}:</strong> ${amount:,.2f}</li>"
        actions_html += f"""
        </ul>
        <p><strong>Total to Invest:</strong> ${total_amount:,.2f}</p>
        <p>Strategy: Buy these amounts now. If FGI drops further, consider additional buys at lower thresholds per your plan.</p>
        """
    elif action_type == "sell":
        actions_html = f"""
        <h3>📉 SELLING OPPORTUNITY DETECTED</h3>
        <h4>Recommended Sell Orders:</h4>
        <ul>
        """
        for coin, amount in actions.items():
            if amount > 0:
                actions_html += f"<li><strong>{coin}:</strong> ${amount:,.2f}</li>"
        actions_html += f"""
        </ul>
        <p><strong>Total to Sell:</strong> ${total_amount:,.2f}</p>
        <p>Strategy: Take profits now. Consider setting stop-loss orders to protect remaining gains.</p>
        """
    else:
        actions_html = """
        <h3>⚖️ MARKET IN NEUTRAL ZONE</h3>
        <p>Recommended action: Hold current positions</p>
        <p>Monitor for FGI &lt;40 (buy) or &gt;70 (sell) opportunities</p>
        """

    # Overall market sentiment
    sentiment_counts = {'Positive': 0, 'Negative': 0, 'Neutral': 0}
    for item in news_items:
        sentiment_counts[item['sentiment']] += 1

    if sentiment_counts['Positive'] > sentiment_counts['Negative']:
        market_sentiment = "POSITIVE TREND 📈"
    elif sentiment_counts['Negative'] > sentiment_counts['Positive']:
        market_sentiment = "NEGATIVE TREND 📉"
    else:
        market_sentiment = "NEUTRAL MARKET ➖"

    # Build the full HTML
    html = f"""
    <html>
        <head>
            <style>
                body {{ font-family: Arial, sans-serif; line-height: 1.6; color: #333; }}
                .container {{ max-width: 800px; margin: 0 auto; padding: 20px; }}
                .header {{ background-color: #f8f9fa; padding: 20px; text-align: center; border-bottom: 1px solid #ddd; }}
                .section {{ margin: 20px 0; padding: 20px; background-color: white; border-radius: 5px; box-shadow: 0 2px 5px rgba(0,0,0,0.1); }}
                table {{ width: 100%; border-collapse: collapse; margin: 15px 0; }}
                th {{ background-color: #f2f2f2; text-align: left; }}
                .positive {{ color: #4CAF50; }}
                .negative {{ color: #F44336; }}
                .neutral {{ color: #9E9E9E; }}
                h1, h2, h3 {{ color: #2c3e50; }}
                .footer {{ text-align: center; margin-top: 30px; color: #7f8c8d; font-size: 12px; }}
            </style>
        </head>
        <body>
            <div class="container">
                <div class="header">
                    <h1>Cryptocurrency Portfolio Strategy Report</h1>
                    <p>Generated on {time.strftime("%Y-%m-%d %H:%M:%S")}</p>
                </div>

                <div class="section">
                    <h2>Market Overview</h2>
                    <p><strong>Fear & Greed Index:</strong> {fgi}</p>
                    <p><strong>Market Analysis:</strong> {sentiment_analysis}</p>
                    <p><strong>Overall Market Sentiment:</strong> {market_sentiment}</p>
                </div>

                <div class="section">
                    <h2>Recent Market News</h2>
                    {news_html}
                </div>

                <div class="section">
                    <h2>Current Prices</h2>
                    {prices_html}
                </div>

                <div class="section">
                    <h2>Investment Strategy</h2>
                    {strategy_table}
                    {actions_html}
                    <p><strong>Note:</strong> INJ uses profit-based selling (50% at 2x, 50% at 3x) not FGI-based</p>
                </div>

                <div class="footer">
                    <p>This report was generated automatically. Please review recommendations before making investment decisions.</p>
                    <p>Cryptocurrency investments are volatile and risky. Only invest what you can afford to lose.</p>
                </div>
            </div>
        </body>
    </html>
    """

    return html

def send_email(html_content):
    """Send the portfolio report via email"""
    try:
        # Create message container
        msg = MIMEMultipart('alternative')
        msg['Subject'] = f"Crypto Portfolio Strategy Report - {time.strftime('%Y-%m-%d')}"
        msg['From'] = EMAIL_USER
        msg['To'] = RECIPIENT_EMAIL

        # Create HTML version of message
        html_part = MIMEText(html_content, 'html')
        msg.attach(html_part)

        # Send email
        with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
            server.starttls()
            server.login(EMAIL_USER, EMAIL_PASSWORD)
            server.sendmail(EMAIL_USER, RECIPIENT_EMAIL, msg.as_string())

        print("\n✅ Email sent successfully!")
        return True
    except Exception as e:
        print(f"\n❌ Error sending email: {e}")
        return False

def main():
    # Get market data
    fgi = get_fear_greed_index()
    if fgi is None:
        print("\n⚠️ Failed to get Fear & Greed Index. Try again later.")
        return

    sentiment = analyze_market(fgi)
    news_items = get_market_news()
    news_items = analyze_news_sentiment(news_items)
    prices = get_crypto_prices(list(COIN_RULES.keys()))

    # Initialize variables for actions
    action_type = "hold"
    actions = {}
    total_amount = 0
    portfolio_size = 0
    holdings = {}

    # Determine investment actions
    if fgi < 40:  # Buying zone
        action_type = "buy"
        portfolio_size = 1000  # Default portfolio size for scheduled tasks
        actions = calculate_buy_amounts(fgi, portfolio_size)
        total_amount = sum(actions.values())
    elif fgi >= 70:  # Selling zone
        action_type = "sell"
        # Default holdings for scheduled tasks
        holdings = {coin: 100 for coin in COIN_RULES}
        actions = calculate_sell_amounts(fgi, holdings)
        total_amount = sum(actions.values())

    # Generate HTML report
    html_report = generate_html_report(
        fgi, sentiment, news_items, prices,
        action_type, actions, total_amount,
        portfolio_size, holdings
    )

    # Send email automatically for scheduled tasks
    send_email(html_report)
    print("\nReport sent via email.")

if __name__ == "__main__":
    print("🚀 CRYPTOCURRENCY PORTFOLIO MANAGER")
    print("🔐 Strategy: Fear & Greed Index-based Trading")
    print("📆 Last update:", time.strftime("%Y-%m-%d %H:%M:%S"))
    print("📰 News powered by CryptoCointracker")
    print("📧 Email functionality enabled")
    main()

🚀 CRYPTOCURRENCY PORTFOLIO MANAGER
🔐 Strategy: Fear & Greed Index-based Trading
📆 Last update: 2025-07-09 19:06:47
📰 News powered by CryptoCointracker
📧 Email functionality enabled

✅ Email sent successfully!

Report sent via email.


In [ ]:
import requests
import time
import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import json

# Configuration with your exact rules
COIN_RULES = {
    'BTC': {
        'allocation': 0.35,
        'buy_tiers': [(40, 140), (30, 140), (20, 70)],
        'sell_rules': [(75, 0.25), (85, 0.50)]
    },
    'ETH': {
        'allocation': 0.25,
        'buy_tiers': [(40, 100), (30, 100), (20, 50)],
        'sell_rules': [(75, 0.25), (85, 0.50)]
    },
    'SOL': {
        'allocation': 0.20,
        'buy_tiers': [(40, 80), (30, 80), (20, 40)],
        'sell_rules': [(70, 0.30), (80, 0.50)]
    },
    'ADA': {
        'allocation': 0.15,
        'buy_tiers': [(40, 60), (30, 60), (20, 30)],
        'sell_rules': [(70, 0.40), (80, 0.50)]
    },
    'INJ': {
        'allocation': 0.05,
        'buy_tiers': [(40, 20), (30, 20), (20, 10)],
        'sell_rules': []  # Profit-based only
    }
}

# Email configuration (set these as environment variables)
EMAIL_USER = os.getenv('EMAIL_USER', 'chanakaworkmail@gmail.com')
EMAIL_PASSWORD = os.getenv('EMAIL_PASSWORD', 'edzm eugn hdgk qinb')
SMTP_SERVER = os.getenv('SMTP_SERVER', 'smtp.gmail.com')
SMTP_PORT = int(os.getenv('SMTP_PORT', 587))
RECIPIENT_EMAIL = os.getenv('RECIPIENT_EMAIL', 'cbcusandaruwan@gmail.com')

# Get API keys from environment variables
ALPHA_VANTAGE_API_KEY = os.getenv('ALPHA_VANTAGE_API_KEY', 'HW34AQWZ25LNK9SA')

def get_fear_greed_index():
    """Fetch current Fear & Greed Index from Alternative.me"""
    url = "https://api.alternative.me/fng/?limit=1"
    try:
        response = requests.get(url, timeout=10)
        data = response.json()
        return int(data['data'][0]['value'])
    except Exception as e:
        print(f"Error fetching Fear & Greed Index: {e}")
        return None

def analyze_market(fgi):
    """Provide detailed market analysis based on FGI score"""
    if fgi >= 85:
        return ("🚨 EXTREME GREED: Market at peak euphoria. Strong sell signal. "
                "Consider taking significant profits as correction likely.")
    elif fgi >= 75:
        return ("📈 Extreme Greed: Market showing greed characteristics. "
                "Good time to take partial profits. Set stop-losses.")
    elif fgi >= 65:
        return ("📈 Greed: Bullish sentiment growing. "
                "Watch for profit-taking opportunities. Avoid FOMO buys.")
    elif fgi >= 55:
        return ("➖ Neutral: Market in balance. "
                "Maintain positions but monitor for trend changes.")
    elif fgi >= 45:
        return ("➖ Neutral: Transition phase. "
                "No strong signals. Good time for research.")
    elif fgi >= 30:
        return ("📉 Fear: Accumulation opportunity. "
                "Good zone for strategic buying. Consider DCA.")
    elif fgi >= 20:
        return ("📉📉 Extreme Fear: Strong fear in market. "
                "Potential buying opportunity for long-term holders.")
    else:
        return ("🚨 PANIC ZONE: Extreme fear and capitulation. "
                "Historically good entry point but high volatility expected.")

def calculate_buy_amounts(fgi, portfolio_size=1000):
    """Calculate exact USD amounts to buy based on FGI and portfolio size"""
    scale = portfolio_size / 1000  # Scale amounts based on portfolio size
    actions = {}

    for coin, rules in COIN_RULES.items():
        amount = 0
        # Find the highest applicable buy tier
        for threshold, tier_amount in rules['buy_tiers']:
            if fgi < threshold:
                amount = tier_amount * scale
                break  # Use the highest matched tier
        actions[coin] = round(amount, 2)

    return actions

def calculate_sell_amounts(fgi, holdings):
    """Calculate USD amounts to sell based on FGI and current holdings"""
    actions = {}

    for coin, amount in holdings.items():
        if coin not in COIN_RULES:
            continue

        sell_rules = COIN_RULES[coin]['sell_rules']
        sell_amount = 0

        # Apply the strongest sell rule that matches
        max_percent = 0
        for threshold, sell_percent in sell_rules:
            if fgi >= threshold and sell_percent > max_percent:
                max_percent = sell_percent

        if max_percent > 0:
            sell_amount = amount * max_percent

        actions[coin] = round(sell_amount, 2)

    return actions

def get_market_news_with_sentiment():
    """Get latest crypto news with sentiment from Alpha Vantage"""
    try:
        url = "https://www.alphavantage.co/query"
        params = {
            'function': 'NEWS_SENTIMENT',
            'tickers': 'CRYPTO:BTC,CRYPTO:ETH',
            'topics': 'blockchain,technology',
            'sort': 'LATEST',
            'limit': '5',
            'apikey': ALPHA_VANTAGE_API_KEY
        }
        response = requests.get(url, params=params, timeout=10)
        data = response.json()
        print("alphavantage",data)
        # Process news items
        news_items = []
        for item in data.get('feed', [])[:3]:
            title = item.get('title', '')
            source = item.get('source', 'Unknown')
            date = item.get('time_published', '')[:10]  # Get YYYY-MM-DD

            # Get overall sentiment score
            sentiment_info = item.get('overall_sentiment_score', 0)
            sentiment_score = float(sentiment_info) if sentiment_info else 0

            # Determine sentiment label and emoji
            if sentiment_score > 0.35:
                sentiment = "👍 STRONGLY POSITIVE"
                emoji = "🟢"
            elif sentiment_score > 0.15:
                sentiment = "👍 Positive"
                emoji = "🟢"
            elif sentiment_score < -0.35:
                sentiment = "👎 STRONGLY NEGATIVE"
                emoji = "🔴"
            elif sentiment_score < -0.15:
                sentiment = "👎 Negative"
                emoji = "🔴"
            else:
                sentiment = "➖ Neutral"
                emoji = "⚪"

            news_items.append({
                'title': title,
                'sentiment': sentiment,
                'score': sentiment_score,
                'source': source,
                'date': date,
                'emoji': emoji
            })

        return news_items

    except Exception as e:
        print(f"News API error: {e}")
        # Fallback news with sentiment
        return [
            {
                'title': "Major institutions increasing Bitcoin investments",
                'sentiment': "👍 STRONGLY POSITIVE",
                'score': 0.8,
                'source': "CryptoBriefing",
                'date': time.strftime("%Y-%m-%d"),
                'emoji': "🟢"
            },
            {
                'title': "Regulatory concerns cause market uncertainty",
                'sentiment': "👎 Negative",
                'score': -0.25,
                'source': "CoinDesk",
                'date': time.strftime("%Y-%m-%d"),
                'emoji': "🔴"
            },
            {
                'title': "Ethereum network upgrade scheduled for next quarter",
                'sentiment': "➖ Neutral",
                'score': 0.05,
                'source': "Decrypt",
                'date': time.strftime("%Y-%m-%d"),
                'emoji': "⚪"
            }
        ]

def get_crypto_prices(coins):
    """Get current prices from CoinGecko"""
    try:
        coin_ids = {
            'BTC': 'bitcoin',
            'ETH': 'ethereum',
            'SOL': 'solana',
            'ADA': 'cardano',
            'INJ': 'injective-protocol'
        }
        ids = ','.join([coin_ids[coin] for coin in coins])
        url = f"https://api.coingecko.com/api/v3/simple/price?ids={ids}&vs_currencies=usd"
        response = requests.get(url, timeout=10)
        prices = response.json()
        return {coin: prices[coin_ids[coin]]['usd'] for coin in coins}
    except:
        # Return placeholder prices if API fails
        return {
            'BTC': 35000,
            'ETH': 2000,
            'SOL': 100,
            'ADA': 0.5,
            'INJ': 25
        }

def generate_html_report(fgi, sentiment_analysis, news_items, prices, action_type, actions, total_amount, portfolio_size=None, holdings=None):
    """Generate HTML report for email"""
    # Generate strategy table
    strategy_table = """
    <table border="1" cellpadding="5" cellspacing="0" width="100%">
        <tr>
            <th>Coin</th>
            <th>Allocation</th>
            <th>Buy Rules (FGI-based)</th>
            <th>Sell Rules (FGI-based)</th>
        </tr>
    """

    for coin, rules in COIN_RULES.items():
        buy_str = ", ".join([f"${amt} at FGI&lt;{th}" for th, amt in rules['buy_tiers']])

        sell_str = ", ".join([f"Sell {int(perc*100)}% at FGI&gt;{th}" for th, perc in rules['sell_rules']])
        if not sell_str and coin == 'INJ':
            sell_str = "Sell 50% at 2x, 50% at 3x (Profit-based)"

        alloc_percent = f"{rules['allocation']*100:.0f}%"

        strategy_table += f"""
        <tr>
            <td>{coin}</td>
            <td>{alloc_percent}</td>
            <td>{buy_str}</td>
            <td>{sell_str}</td>
        </tr>
        """

    strategy_table += "</table>"

    # Generate news section
    news_html = ""
    for item in news_items[:3]:
        news_html += f"""
        <div style="margin-bottom: 15px; padding: 10px; border-left: 4px solid {'#4CAF50' if 'POSITIVE' in item['sentiment'] else '#F44336' if 'NEGATIVE' in item['sentiment'] else '#9E9E9E'};">
            <div style="font-weight: bold; font-size: 16px;">{item['emoji']} {item['title']}</div>
            <div style="color: #666; font-size: 14px;">{item['sentiment']} | Source: {item['source']} | Date: {item['date']}</div>
        </div>
        """

    # Generate prices section
    prices_html = "<ul>"
    for coin, price in prices.items():
        prices_html += f"<li><strong>{coin}:</strong> ${price:,.2f}</li>"
    prices_html += "</ul>"

    # Generate action recommendations
    actions_html = ""
    if action_type == "buy":
        actions_html = f"""
        <h3>✅ BUYING OPPORTUNITY DETECTED</h3>
        <p>Portfolio Size: ${portfolio_size:,.2f}</p>
        <h4>Recommended Buy Orders:</h4>
        <ul>
        """
        for coin, amount in actions.items():
            if amount > 0:
                actions_html += f"<li><strong>{coin}:</strong> ${amount:,.2f}</li>"
        actions_html += f"""
        </ul>
        <p><strong>Total to Invest:</strong> ${total_amount:,.2f}</p>
        <p>Strategy: Buy these amounts now. If FGI drops further, consider additional buys at lower thresholds per your plan.</p>
        """
    elif action_type == "sell":
        actions_html = f"""
        <h3>📉 SELLING OPPORTUNITY DETECTED</h3>
        <h4>Recommended Sell Orders:</h4>
        <ul>
        """
        for coin, amount in actions.items():
            if amount > 0:
                actions_html += f"<li><strong>{coin}:</strong> ${amount:,.2f}</li>"
        actions_html += f"""
        </ul>
        <p><strong>Total to Sell:</strong> ${total_amount:,.2f}</p>
        <p>Strategy: Take profits now. Consider setting stop-loss orders to protect remaining gains.</p>
        """
    else:
        actions_html = """
        <h3>⚖️ MARKET IN NEUTRAL ZONE</h3>
        <p>Recommended action: Hold current positions</p>
        <p>Monitor for FGI &lt;40 (buy) or &gt;70 (sell) opportunities</p>
        """

    # Overall market sentiment
    if news_items:
        avg_sentiment = sum(item['score'] for item in news_items) / len(news_items)
        if avg_sentiment > 0.35:
            market_sentiment = "STRONGLY BULLISH 📈📈"
        elif avg_sentiment > 0.15:
            market_sentiment = "BULLISH 📈"
        elif avg_sentiment < -0.35:
            market_sentiment = "STRONGLY BEARISH 📉📉"
        elif avg_sentiment < -0.15:
            market_sentiment = "BEARISH 📉"
        else:
            market_sentiment = "NEUTRAL ➖"
    else:
        market_sentiment = "Not available"

    # Build the full HTML
    html = f"""
    <html>
        <head>
            <style>
                body {{ font-family: Arial, sans-serif; line-height: 1.6; color: #333; }}
                .container {{ max-width: 800px; margin: 0 auto; padding: 20px; }}
                .header {{ background-color: #f8f9fa; padding: 20px; text-align: center; border-bottom: 1px solid #ddd; }}
                .section {{ margin: 20px 0; padding: 20px; background-color: white; border-radius: 5px; box-shadow: 0 2px 5px rgba(0,0,0,0.1); }}
                table {{ width: 100%; border-collapse: collapse; margin: 15px 0; }}
                th {{ background-color: #f2f2f2; text-align: left; }}
                .positive {{ color: #4CAF50; }}
                .negative {{ color: #F44336; }}
                .neutral {{ color: #9E9E9E; }}
                h1, h2, h3 {{ color: #2c3e50; }}
                .footer {{ text-align: center; margin-top: 30px; color: #7f8c8d; font-size: 12px; }}
            </style>
        </head>
        <body>
            <div class="container">
                <div class="header">
                    <h1>Cryptocurrency Portfolio Strategy Report</h1>
                    <p>Generated on {time.strftime("%Y-%m-%d %H:%M:%S")}</p>
                </div>

                <div class="section">
                    <h2>Market Overview</h2>
                    <p><strong>Fear & Greed Index:</strong> {fgi}</p>
                    <p><strong>Market Analysis:</strong> {sentiment_analysis}</p>
                    <p><strong>Overall Market Sentiment:</strong> {market_sentiment}</p>
                </div>

                <div class="section">
                    <h2>Recent Market News</h2>
                    {news_html}
                </div>

                <div class="section">
                    <h2>Current Prices</h2>
                    {prices_html}
                </div>

                <div class="section">
                    <h2>Investment Strategy</h2>
                    {strategy_table}
                    {actions_html}
                    <p><strong>Note:</strong> INJ uses profit-based selling (50% at 2x, 50% at 3x) not FGI-based</p>
                </div>

                <div class="footer">
                    <p>This report was generated automatically. Please review recommendations before making investment decisions.</p>
                    <p>Cryptocurrency investments are volatile and risky. Only invest what you can afford to lose.</p>
                </div>
            </div>
        </body>
    </html>
    """

    return html

def send_email(html_content):
    """Send the portfolio report via email"""
    try:
        # Create message container
        msg = MIMEMultipart('alternative')
        msg['Subject'] = f"Crypto Portfolio Strategy Report - {time.strftime('%Y-%m-%d')}"
        msg['From'] = EMAIL_USER
        msg['To'] = RECIPIENT_EMAIL

        # Create HTML version of message
        html_part = MIMEText(html_content, 'html')
        msg.attach(html_part)

        # Send email
        with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
            server.starttls()
            server.login(EMAIL_USER, EMAIL_PASSWORD)
            server.sendmail(EMAIL_USER, RECIPIENT_EMAIL, msg.as_string())

        print("\n✅ Email sent successfully!")
        return True
    except Exception as e:
        print(f"\n❌ Error sending email: {e}")
        return False

def main():
    # Get market data
    fgi = get_fear_greed_index()
    if fgi is None:
        print("\n⚠️ Failed to get Fear & Greed Index. Try again later.")
        return

    sentiment = analyze_market(fgi)
    news_items = get_market_news_with_sentiment()
    prices = get_crypto_prices(list(COIN_RULES.keys()))

    # Initialize variables for actions
    action_type = "hold"
    actions = {}
    total_amount = 0
    portfolio_size = 0
    holdings = {}

    # Determine investment actions
    if fgi < 40:  # Buying zone
        action_type = "buy"
        portfolio_size = float(input("\nEnter your total planned portfolio size (USD): ") or 1000)
        actions = calculate_buy_amounts(fgi, portfolio_size)
        total_amount = sum(actions.values())
    elif fgi >= 70:  # Selling zone
        action_type = "sell"
        print("\nEnter your current holdings (USD value):")
        for coin in COIN_RULES:
            holdings[coin] = float(input(f"  {coin} amount: ") or 0)
        actions = calculate_sell_amounts(fgi, holdings)
        total_amount = sum(actions.values())

    # Generate HTML report
    html_report = generate_html_report(
        fgi, sentiment, news_items, prices,
        action_type, actions, total_amount,
        portfolio_size, holdings
    )

    # Save HTML report to file
    with open("crypto_report.html", "w") as f:
        f.write(html_report)
    print("\n📄 Report saved as crypto_report.html")

    # send_email(html_report)

if __name__ == "__main__":
    print("🚀 CRYPTOCURRENCY PORTFOLIO MANAGER")
    print("🔐 Strategy: Fear & Greed Index-based Trading")
    print("📆 Last update:", time.strftime("%Y-%m-%d %H:%M:%S"))
    print("📰 News powered by Alpha Vantage API")
    print("📧 Email functionality enabled")
    main()

🚀 CRYPTOCURRENCY PORTFOLIO MANAGER
🔐 Strategy: Fear & Greed Index-based Trading
📆 Last update: 2025-07-09 18:52:41
📰 News powered by Alpha Vantage API
📧 Email functionality enabled
alphavantage {'items': '50', 'sentiment_score_definition': 'x <= -0.35: Bearish; -0.35 < x <= -0.15: Somewhat-Bearish; -0.15 < x < 0.15: Neutral; 0.15 <= x < 0.35: Somewhat_Bullish; x >= 0.35: Bullish', 'relevance_score_definition': '0 < x <= 1, with a higher score indicating higher relevance.', 'feed': [{'title': 'Bitget Integrates xStocks to Bring Wall Street to Web3 with Tokenized Stocks on Onchain', 'url': 'https://www.benzinga.com/pressreleases/25/07/g46316381/bitget-integrates-xstocks-to-bring-wall-street-to-web3-with-tokenized-stocks-on-onchain', 'time_published': '20250709T120743', 'authors': ['Globe Newswire'], 'summary': 'VICTORIA, Seychelles, July 09, 2025 ( GLOBE NEWSWIRE ) -- Bitget, the leading cryptocurrency exchange and Web3 company, has introduced support for tokenized stocks via its Onchain

In [ ]:
url = "https://bitcoinnews.com/wp-json/wp/v2/posts?per_page=3"
response = requests.get(url, timeout=5)
data = response.json()
print("\nbitcoinnew s",[item['title']['rendered'] for item in data[:3]])
print("\nbitcoinnew s",[item['link'] for item in data[:3]])
print("\nbitcoinnew s",[item['excerpt']['rendered'] for item in data[:3]])




bitcoinnew s ['Remixpoint Becomes First Japan-Listed Company to Pay CEO in Bitcoin', 'Elon Musk Launches ‘America Party,’ Puts Bitcoin at the Center of US Politics', 'Texas Weather Crises and Grid Strain Slam Bitcoin Mining in June']

bitcoinnew s ['https://bitcoinnews.com/adoption/remixpoint-japan-pay-ceo-in-bitcoin/', 'https://bitcoinnews.com/adoption/elon-musk-america-party-bitcoin-embrace/', 'https://bitcoinnews.com/mining/texas-bitcoin-mining-weather-crises-june/']

bitcoinnew s ['<p>Remixpoint has built a strong digital asset portfolio, and currently holds over 1,000 BTC in its coffers.</p>\n', '<p>Elon Musk confirmed on X that Bitcoin will play a big role in his new political party, &#8220;America Party&#8221;, calling fiat &#8220;hopeless&#8221;.</p>\n', '<p>Latest heatwaves, power curtailments, and floods caused Texas bitcoin mining operations to take a hit, dropping the network&#8217;s hashrate by 10%.</p>\n']


In [ ]:
def get_sentiment_score(text):
    url = "https://bitcoinnews.com/wp-json/wp/v2/posts?per_page=3"
    response = requests.get(url, timeout=5)
    data = response.json()


    """Analyze sentiment of news text using TextRazor API"""
    try:
        headers = {'X-TextRazor-Key': "57944a378e79557805eb7553731db6a68a85125a06683b987f109aad"}
        params = {'extractors': 'entities,sentiment', 'text': text}
        response = requests.post('https://api.textrazor.com/', headers=headers, data=params)
        data = response.json()

        # Get sentiment score if available
        sentiment = data.get('response', {}).get('sentiment', None)
        print(data['response'])
        if sentiment:
            return float(sentiment)

    except Exception as e:
        print(f"Sentiment analysis error: {e}")

    return 0  # Neutral if no sentiment detected


print(get_sentiment_score("Remixpoint has built a strong digital asset portfolio, and currently holds over 1,000 BTC in its coffers."))

{'language': 'eng', 'languageIsReliable': True, 'entities': [{'id': 0, 'type': ['Number'], 'matchingTokens': [13], 'entityId': '1000', 'confidenceScore': 0.5, 'wikiLink': '', 'matchedText': '1,000', 'relevanceScore': 0, 'entityEnglishId': '', 'startingPos': 80, 'endingPos': 85}, {'id': 1, 'matchingTokens': [17], 'entityId': 'Coffer', 'confidenceScore': 1.857, 'wikiLink': 'http://en.wikipedia.org/wiki/Coffer', 'matchedText': 'coffers', 'freebaseId': '/m/055h_x', 'relevanceScore': 0, 'entityEnglishId': 'Coffer', 'startingPos': 97, 'endingPos': 104, 'wikidataId': 'Q640104'}, {'id': 2, 'type': ['Company'], 'matchingTokens': [0], 'entityId': 'Remixpoint', 'freebaseTypes': ['/organization/organization'], 'confidenceScore': 0.5, 'wikiLink': '', 'matchedText': 'Remixpoint', 'relevanceScore': 0, 'entityEnglishId': '', 'startingPos': 0, 'endingPos': 10}]}
0


In [ ]:
url = "https://bitcoinnews.com/wp-json/wp/v2/posts?per_page=3"
response = requests.get(url, timeout=5)
data = response.json()

# get_sentiment_score()
# print("\nbitcoinnew s",[item['title']['rendered'] for item in data[:3]])
print("\nbitcoinnew s",[item['link'] for item in data[:1]])
# print([item['excerpt']['rendered'] for item in data[:1]][0])
# get_sentiment_score([item['content']['rendered'] for item in data[:1]][0])


bitcoinnew s ['https://bitcoinnews.com/adoption/remixpoint-japan-pay-ceo-in-bitcoin/']


In [ ]:
print(get_sentiment_score("Remixpoint Becomes First Japan-Listed Company to Pay CEO in Bitcoin"))

0


In [ ]:
import requests

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=AAPL&apikey=HW34AQWZ25LNK9SA'
# ITAPJIQ80TVGWFCH
r = requests.get(url)
data = r.json()

print(data)

{'items': '50', 'sentiment_score_definition': 'x <= -0.35: Bearish; -0.35 < x <= -0.15: Somewhat-Bearish; -0.15 < x < 0.15: Neutral; 0.15 <= x < 0.35: Somewhat_Bullish; x >= 0.35: Bullish', 'relevance_score_definition': '0 < x <= 1, with a higher score indicating higher relevance.', 'feed': [{'title': "3 Big Surprises Just Hit The Markets - Here's How I'm Trading Them - Apple  ( NASDAQ:AAPL ) , Amazon.com  ( NASDAQ:AMZN ) ", 'url': 'https://www.benzinga.com/news/25/07/46327770/3-big-surprises-just-hit-the-markets-heres-how-im-trading-them', 'time_published': '20250709T175231', 'authors': ['Tom Gentile'], 'summary': "In just the first few days of July, we've seen more market-shaking headlines than we usually get over the span of a full quarter. Congress officially passed President Trump's 'Big Beautiful Bill' last week.", 'banner_image': 'https://cdn.benzinga.com/files/images/story/2025/07/09/TRUMP-MUSK-FEUD.jpeg?width=1200&height=800&fit=crop', 'source': 'Benzinga', 'category_within_so

In [ ]:
def get_market_news():
    """Get reliable crypto news without API key restrictions"""
    try:
        # Get today's date and yesterday's date
        today = datetime.now().strftime("%Y-%m-%d")
        yesterday = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d")

        # Try CryptoCointracker API
        url = "https://cryptocointracker.com/api/news"
        response = requests.get(url, timeout=10)
        news_data = response.json()
        print("news_data", news_data)
        # Filter for recent news (last 2 days)
        recent_news = []
        for item in news_data.get('news', []):
            if item.get('date', '').startswith(today) or item.get('date', '').startswith(yesterday):
                recent_news.append({
                    'title': item.get('title', ''),
                    'source': item.get('source', 'Unknown'),
                    'date': item.get('date', '')[:10],  # Get YYYY-MM-DD
                    'sentiment': "Neutral"  # Placeholder, we'll analyze below
                })

        # Return top 3 recent news
        return recent_news[:3]
    except:
        # Fallback to Bitcoin News API
        try:
            url = "https://api.bitcoinnews.com/api/v1/posts/?limit=3"

            print(url)
            response = requests.get(url, timeout=10)
            print("response",response)
            data = response.json()
            print("bitcoinnews", data)
            news_items = []
            for item in data.get('results', [])[:3]:
                news_items.append({
                    'title': item.get('title', ''),
                    'source': "BitcoinNews",
                    'date': datetime.now().strftime("%Y-%m-%d"),
                    'sentiment': "Neutral"
                })
            return news_items
        except Exception as e:
            # Final fallback - generate placeholder news
            return [
                {
                    'title': "Bitcoin volatility increases as market sentiment shifts",
                    'source': "CryptoCointracker",
                    'date': datetime.now().strftime("%Y-%m-%d"),
                    'sentiment': "Neutral"
                },
                {
                    'title': "Institutional adoption of crypto continues to grow",
                    'source': "CryptoCointracker",
                    'date': datetime.now().strftime("%Y-%m-%d"),
                    'sentiment': "Positive"
                },
                {
                    'title': "Regulatory developments shaping crypto landscape",
                    'source': "CryptoCointracker",
                    'date': datetime.now().strftime("%Y-%m-%d"),
                    'sentiment': "Neutral"
                }
            ]

def analyze_news_sentiment(news_items):
    """Simple sentiment analysis based on keywords"""
    positive_keywords = ['bullish', 'adopt', 'growth', 'approve', 'positive', 'rise', 'gain', 'increase']
    negative_keywords = ['bearish', 'regulat', 'ban', 'negative', 'fall', 'drop', 'decline', 'warn', 'risk']

    for item in news_items:
        title = item['title'].lower()
        positive_count = sum(1 for word in positive_keywords if word in title)
        negative_count = sum(1 for word in negative_keywords if word in title)

        if positive_count > negative_count:
            item['sentiment'] = "Positive"
            item['emoji'] = "🟢"
        elif negative_count > positive_count:
            item['sentiment'] = "Negative"
            item['emoji'] = "🔴"
        else:
            item['sentiment'] = "Neutral"
            item['emoji'] = "⚪"

    return news_items

get_market_news()

https://api.bitcoinnews.com/api/v1/posts/?limit=3


[{'title': 'Bitcoin volatility increases as market sentiment shifts',
  'source': 'CryptoCointracker',
  'date': '2025-07-10',
  'sentiment': 'Neutral'},
 {'title': 'Institutional adoption of crypto continues to grow',
  'source': 'CryptoCointracker',
  'date': '2025-07-10',
  'sentiment': 'Positive'},
 {'title': 'Regulatory developments shaping crypto landscape',
  'source': 'CryptoCointracker',
  'date': '2025-07-10',
  'sentiment': 'Neutral'}]

In [ ]:
api_key="api_live_zSY4VwxPC9VkZUzZtxREjM0KTDfrdyWUXWJlQKSlzIHf"

response = requests.get("https://newsdata.io/api/1/latest?apikey=pub_a9c7d3781a50489f984530372a8b8105&q=bitcoin", timeout=10)
news_data = response.json()
news_items = []
for item in news_data.get('results', [])[:1]:
                print(item)
                # news_items.append({
                #     'title': item.get('title', ''),
                #     'source': "BitcoinNews",
                #     'date': datetime.now().strftime("%Y-%m-%d"),
                #     'sentiment': "Neutral",

                # })

{'article_id': '93ae5c6c343d296139e9a2fc81709765', 'title': 'Genius Act: This new US cryptocurrency law could pave the way for the next global financial crisis', 'link': 'https://techxplore.com/news/2025-07-genius-cryptocurrency-law-pave-global.html', 'keywords': ['business'], 'creator': None, 'description': 'On June 17, the US Senate passed the GENIUS Act, which was seen as a big win for the cryptocurrency sector. The bill aims to regulate a type of cryptocurrency known as "stablecoins," but a closer look at the law reveals that it could, quite easily, lead to the next global economic crash.', 'content': 'ONLY AVAILABLE IN PAID PLANS', 'pubDate': '2025-07-09 17:50:02', 'pubDateTZ': 'UTC', 'image_url': 'https://scx1.b-cdn.net/csz/news/tmb/2022/bitcoin.jpg', 'video_url': None, 'source_id': 'techxplore', 'source_name': 'Tech Xplore', 'source_priority': 236693, 'source_url': 'https://techxplore.com', 'source_icon': 'https://n.bytvi.com/techxplore.png', 'language': 'english', 'country': ['

In [ ]:
news_items

[{'title': "How To Use DRML Miner's Cloud Mining Strategy: A Simple Guide With Direction And Goals",
  'source': 'BitcoinNews',
  'date': '2025-07-10',
  'sentiment': 'Neutral'},
 {'title': 'MyBTCX.com Enables Apple Pay for Global Crypto Purchases',
  'source': 'BitcoinNews',
  'date': '2025-07-10',
  'sentiment': 'Neutral'},
 {'title': '800 miliardi di crypto: Visa è vicina, il futuro di più',
  'source': 'BitcoinNews',
  'date': '2025-07-10',
  'sentiment': 'Neutral'}]

In [ ]:
import requests

url = "https://api.apitube.io/v1/news/everything?api_key=api_live_zSY4VwxPC9VkZUzZtxREjM0KTDfrdyWUXWJlQKSlzIHf&per_page=1"

querystring = {"per_page=10"}

querystring = {
    "sentiment.overall.score.min": "0.7",
    "country.code":"us",
    "category.id":"medtop:04000000",
    "topic.id":"industry.crypto_news",
    "industry.id": "409",
    "sort.by": "sentiment.overall.score",
    "sort.order": "desc",
    "api_key": "api_live_zSY4VwxPC9VkZUzZtxREjM0KTDfrdyWUXWJlQKSlzIHf",
    "per_page":1
}

response = requests.request("GET", "https://api.apitube.io/v1/news/everything", params=querystring)

print(response.text)

{"status":"ok","limit":1,"path":"https://api.apitube.io/v1/news/everything?sentiment.overall.score.min=0.7\u0026topic.id=industry.crypto_news\u0026industry.id=409\u0026sort.by=sentiment.overall.score\u0026sort.order=desc\u0026api_key=api_live_zSY4VwxPC9VkZUzZtxREjM0KTDfrdyWUXWJlQKSlzIHf\u0026per_page=1","page":1,"has_next_pages":true,"next_page":"https://api.apitube.io/v1/news/everything?api_key=api_live_zSY4VwxPC9VkZUzZtxREjM0KTDfrdyWUXWJlQKSlzIHf\u0026industry.id=409\u0026page=2\u0026per_page=1\u0026sentiment.overall.score.min=0.7\u0026sort.by=sentiment.overall.score\u0026sort.order=desc\u0026topic.id=industry.crypto_news","has_previous_page":false,"previous_page":"","export":{"json":"https://api.apitube.io/v1/news/everything?api_key=api_live_zSY4VwxPC9VkZUzZtxREjM0KTDfrdyWUXWJlQKSlzIHf\u0026export=json\u0026industry.id=409\u0026per_page=1\u0026sentiment.overall.score.min=0.7\u0026sort.by=sentiment.overall.score\u0026sort.order=desc\u0026topic.id=industry.crypto_news","xlsx":"https:/

In [ ]:
news_data = response.json()
news_items = []
for item in news_data.get('results', [])[:1]:
                print(item['sentiment']['overall']['polarity'])
                # news_items.append({
                #     'title': item.get('title', ''),
                #     'source': "BitcoinNews",
                #     'date': datetime.now().strftime("%Y-%m-%d"),
                #     'sentiment': "Neutral",

                # })

positive
